# 🩺 U-Net: Precyzyjna Segmentacja Obrazu

Klasyfikacja mówi: "Tu jest kot".
Detekcja mówi: "Kot jest w tym pudełku".
**Segmentacja** mówi: "Te piksele to kot, a tamte to tło".

Architektura U-Net to standard w medycynie. Składa się z:
1.  **Zjeżdżalni w dół (Encoder):** Używa `Conv2d` i `MaxPool`, żeby zrozumieć *co jest na zdjęciu* (ale traci informację o lokalizacji, bo obrazek robi się mały).
2.  **Wspinaczki w górę (Decoder):** Używa `ConvTranspose2d` (odwrotny splot), żeby powiększyć obrazek z powrotem do oryginalnego rozmiaru.
3.  **Skrótów (Skip Connections):** Kopiujemy obrazek z lewej strony na prawą. Dzięki temu sieć pamięta, gdzie były krawędzie.

Zbudujemy tę architekturę od zera w PyTorch.

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

# 1. BLOK PODSTAWOWY (Double Conv)
# W U-Net zawsze robimy dwie konwolucje po sobie.
# Conv3x3 -> ReLU -> Conv3x3 -> ReLU

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), # Stabilizacja
            nn.ReLU(inplace=True),
            
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

print("Podstawowy klocek gotowy.")

Podstawowy klocek gotowy.


## Budowa U-Net

To jest serce modelu.
*   **Downs:** Lista warstw idących w dół (zwiększamy liczbę filtrów: 64 -> 128 -> 256...).
*   **Ups:** Lista warstw idących w górę (zmniejszamy liczbę filtrów: ...256 -> 128 -> 64).
*   **Bottleneck:** Najniższy punkt litery U.

Najważniejsza linijka w kodzie to `torch.cat`. To ona łączy (skleja) obrazek z Encodera z obrazkiem z Decodera.

In [2]:
class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super().__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 1. Budujemy Zjeżdżalnię (Down)
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # 2. Budujemy Wspinaczkę (Up)
        for feature in reversed(features):
            # ConvTranspose2d powiększa obrazek (2x2 -> 4x4)
            self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))

        # 3. Najniższy punkt (Bottleneck)
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        
        # 4. Ostatnia warstwa (Mapowanie na wynik 1x1)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # --- W DÓŁ ---
        for down in self.downs:
            x = down(x)
            skip_connections.append(x) # Zapisujemy obrazek na później (do skrótu)
            x = self.pool(x)

        # Dno sieci
        x = self.bottleneck(x)
        
        # Odwracamy listę skrótów, żeby brać je w dobrej kolejności
        skip_connections = skip_connections[::-1]

        # --- W GÓRĘ ---
        # Iterujemy co 2 kroki (bo mamy Transpose + DoubleConv)
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x) # Powiększamy
            
            skip_connection = skip_connections[idx//2] # Bierzemy pasujący skrót
            
            # --- MAGIA: Sklejamy (Concatenate) ---
            # Jeśli wymiary się nie zgadzają (np. przez nieparzyste dzielenie), trzeba przyciąć.
            # Tutaj dla uproszczenia zakładamy idealne wymiary (potęgi 2).
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1) # Sklejamy wzdłuż kanałów
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

print("Architektura U-Net zdefiniowana.")

Architektura U-Net zdefiniowana.


## Test Wymiarów (Smoke Test)

Zanim zaczniemy trenować na prawdziwych zdjęciach (co trwa godziny), sprawdźmy, czy matematyka się zgadza.
Wrzucimy losowy szum o wymiarach `160x160`.
Oczekujemy, że sieć zwróci `160x160` (mapę segmentacji). Jeśli zwróci inny rozmiar -> mamy błąd w kodzie.

In [3]:
def test_unet():
    # Losowy obrazek: Batch=3, Kanały=1 (czarno-biały), 160x160 pikseli
    x = torch.randn((3, 1, 160, 160))
    
    # Tworzymy model
    model = UNET(in_channels=1, out_channels=1)
    
    # Przepuszczamy dane
    preds = model(x)
    
    print(f"Wejście: {x.shape}")
    print(f"Wyjście: {preds.shape}")
    
    assert preds.shape == x.shape
    print("✅ SUKCES! Wymiary wejścia i wyjścia są identyczne.")

test_unet()

Wejście: torch.Size([3, 1, 160, 160])
Wyjście: torch.Size([3, 1, 160, 160])
✅ SUKCES! Wymiary wejścia i wyjścia są identyczne.


## 🧠 Podsumowanie: Po co te skróty (Skip Connections)?

Dlaczego U-Net jest lepszy od zwykłego Autoenkodera?

**Tu jest haczyk.**
Kiedy Autoenkoder zmniejsza zdjęcie do malutkiego wektora (Bottleneck), traci informację o **precyzyjnych krawędziach**. Wie, że na zdjęciu jest "płuco", ale nie wie dokładnie, gdzie kończy się jego granica (piksel 120 czy 121?).

**Skip Connections** działają jak kalka techniczna.
Decoder próbuje narysować płuco z pamięci (z Bottlenecku), ale dostaje też "ściągę" z Encodera – oryginalny obraz w wysokiej rozdzielczości z danej warstwy.
Łączy te dwie informacje:
*   Kontekst ("To jest płuco") z dołu.
*   Lokalizację ("Krawędź jest tutaj") ze skrótu.

Dlatego U-Net daje ostre jak brzytwa maski segmentacji.